In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from scipy.optimize import minimize
from isic_helper import get_folds
from isic_helper import compute_pauc, compute_auc

In [2]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_preds_ensemble"

model_names = ["cb", "lgb", "resnet18", "efficientnet_b0"]
versions = ["v1", "v3", "v2",  "v1"]
paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-train" for model_name, version in zip(model_names, versions)]

# weights = [0.4, 0.6]

In [3]:
val_auc_scores = {}
val_pauc_scores = {}
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    oof_preds_model_df = pd.read_csv(f"{path}/oof_preds_{model_name}_{version}.csv")
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, f"oof_{model_name}_{version}"]],
                                          on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    val_auc_scores[f"{model_name}_{version}"] = {}
    val_pauc_scores[f"{model_name}_{version}"] = {}
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}

In [4]:
oof_columns = [col for col in oof_preds_df.columns if col.startswith("oof")]
all_folds = np.unique(oof_preds_df["fold"])

In [5]:
def pauc_min_func(weights):
    score = 0
    for fold in all_folds:
        fold_ensemble_preds = 0
        for weight, oof_column in zip(weights, oof_columns):
            fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
        score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
    return score / len(all_folds)


print("\n Finding Blending Weights ...")
res_list = []
weights_list = []

for k in range(25):
    starting_values = np.random.uniform(size=len(oof_columns))
    bounds = [(0, 1)] * len(oof_columns)

    res = minimize(
        pauc_min_func,
        starting_values,
        method="L-BFGS-B",
        bounds=bounds,
        options={"disp": False, "maxiter": 10000},
    )

    res_list.append(res["fun"])
    weights_list.append(res["x"])

    
    res_fun = format(res["fun"], '.7f')
    items = [format(item, '.7f') for item in res["x"]]
    
    print(
        "{iter}\tScore: {score}\tWeights: {weights}".format(
            iter=(k + 1),
            score=res_fun,
            weights="\t".join([str(item) for item in items]),
        )
    )

bestSC = np.max(res_list)
bestWght = weights_list[np.argmax(res_list)]
weights = bestWght
blend_score = round(bestSC, 6)


 Finding Blending Weights ...
1	Score: 0.1696483	Weights: 0.2319379	0.3010766	0.8210658	0.4365594
2	Score: 0.1735270	Weights: 0.5479680	0.9037942	0.6138667	0.6064059
3	Score: 0.1664955	Weights: 0.1814057	0.2818139	0.3669577	0.8933276
4	Score: 0.1733896	Weights: 0.1619063	0.0484663	0.0853254	0.0773987
5	Score: 0.1722953	Weights: 0.3011975	0.6666006	0.5546548	0.6356909
6	Score: 0.1680137	Weights: 0.2312116	0.3428620	0.1973287	0.8289964
7	Score: 0.1738046	Weights: 0.4454367	0.7118336	0.4379357	0.3951178
8	Score: 0.1729155	Weights: 0.0097460	0.7118104	0.3093803	0.1541039
9	Score: 0.1727922	Weights: 0.4056355	0.0877305	0.1510437	0.0424878
10	Score: 0.1732157	Weights: 0.9538019	0.9582868	0.2490381	0.3534770
11	Score: 0.1685115	Weights: 0.0291174	0.3564917	0.6838670	0.0352139
12	Score: 0.1664324	Weights: 0.1042399	0.3555494	0.8980239	0.8658948
13	Score: 0.1713538	Weights: 0.3697954	0.1026225	0.5316526	0.0211013
14	Score: 0.1699791	Weights: 0.3767268	0.4704500	0.4945075	0.9663342
15	Score: 0.

In [6]:
print('\n Ensemble Score: {best_score}'.format(best_score=bestSC))
print('\n Best Weights: {weights}'.format(weights=bestWght))
weights = bestWght


 Ensemble Score: 0.17381342225549412

 Best Weights: [0.65857017 0.75441602 0.63078942 0.45994754]


In [7]:
all_folds = np.unique(oof_preds_df["fold"])
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, weight in zip(model_names, versions, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, f"oof_{model_name}_{version}"]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, weight in zip(model_names, versions, weights):
    print(f"Model: {model_name}_{version} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")

Model: cb_v1 | Weightage: 0.6585701655603641
Val AUC scores:
{'fold_1': 0.9523881910669536,
 'fold_2': 0.9535923162036922,
 'fold_3': 0.9457296489498446,
 'fold_4': 0.9721251762738515,
 'fold_5': 0.9470801267252167}
Val PAUC scores:
{'fold_1': 0.1645271757829003,
 'fold_2': 0.1669476383467947,
 'fold_3': 0.15652509640463735,
 'fold_4': 0.17601833248056303,
 'fold_5': 0.15814944833701147}
CV AUC OOF: 0.9524292434775488
CV PAUC OOF: 0.16241203053404624
CV AUC AVG: 0.9541830918439118
CV PAUC AVG: 0.16443353827038137
CV AUC STD: 0.009459406837513047
CV PAUC STD: 0.006965201549198192


Model: lgb_v3 | Weightage: 0.7544160249877135
Val AUC scores:
{'fold_1': 0.9576815612354752,
 'fold_2': 0.9508534742767404,
 'fold_3': 0.9506974516728857,
 'fold_4': 0.9630832636557638,
 'fold_5': 0.9616494948942257}
Val PAUC scores:
{'fold_1': 0.16882108869081178,
 'fold_2': 0.16124885048313303,
 'fold_3': 0.16251325953750728,
 'fold_4': 0.1684323170807802,
 'fold_5': 0.17159456378130328}
CV AUC OOF: 0.95469

In [8]:
oof_preds_df

,isic_id,patient_id,fold,target,oof_cb_v1,oof_lgb_v3,oof_resnet18_v2,oof_efficientnet_b0_v1,oof_preds_ensemble
0,ISIC_0015670,IP_1235828,4,0,0.000104,0.000015,0.019822,0.013287,1.084848
1,ISIC_0015845,IP_8170065,1,0,0.456523,0.652166,0.481737,0.216192,2.416333
2,ISIC_0015864,IP_6724798,5,0,0.000034,0.000019,0.001913,0.001323,0.666902
3,ISIC_0015902,IP_4111386,2,0,0.000022,0.000024,0.021381,0.000164,1.204400
4,ISIC_0024200,IP_8313778,1,0,0.000522,0.000160,0.026625,0.025489,1.825969
...,...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,IP_1140263,3,0,0.003126,0.002748,0.092435,0.011951,2.263647
401055,ISIC_9999951,IP_5678181,3,0,0.000053,0.000030,0.000728,0.000092,0.614461
401056,ISIC_9999960,IP_0076153,2,0,0.001993,0.000537,0.002350,0.000105,1.655776
401057,ISIC_9999964,IP_5231513,5,0,0.000071,0.000028,0.002016,0.002244,0.933035


In [9]:
oof_preds_df.pivot_table(index="target", values=oof_columns+["oof_preds_ensemble"], aggfunc="mean")

,oof_cb_v1,oof_efficientnet_b0_v1,oof_lgb_v3,oof_preds_ensemble,oof_resnet18_v2
target,,,,,
0,0.000671,0.068609,0.000313,1.250787,0.086808
1,0.034607,0.577327,0.021116,2.362855,0.611987


In [10]:
oof_preds_df[oof_columns+["oof_preds_ensemble"]].corr()

,oof_cb_v1,oof_lgb_v3,oof_resnet18_v2,oof_efficientnet_b0_v1,oof_preds_ensemble
oof_cb_v1,1.000000,0.757476,0.230135,0.236565,0.198538
oof_lgb_v3,0.757476,1.000000,0.170230,0.169802,0.130837
oof_resnet18_v2,0.230135,0.170230,1.000000,0.663187,0.607221
oof_efficientnet_b0_v1,0.236565,0.169802,0.663187,1.000000,0.538668
oof_preds_ensemble,0.198538,0.130837,0.607221,0.538668,1.000000
